In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.max_columns', None)

In [2]:
ROOT = "/media/school/project/data.census.gov_downloads"

In [3]:
# Read 2018 gaz
gaz = pd.read_csv(os.path.join(ROOT, '2018_Gaz_tracts_national.txt'), sep='\t')
print(gaz.info())
gaz.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 8 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   USPS                                                                                                                                    74001 non-null  object 
 1   GEOID                                                                                                                                   74001 non-null  int64  
 2   ALAND                                                                                                                                   74001 non-null  int64  
 3   AWATER                                                        

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338
1,AL,1001020200,3325679,5669,1.284,0.002,32.475758,-86.472468
2,AL,1001020300,5349273,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6384276,8408,2.465,0.003,32.471030,-86.444835
4,AL,1001020500,11408869,43534,4.405,0.017,32.458922,-86.421826


In [4]:
# iterate over downloaded data and build dfs from the csvs
# dataframes stored in dfs dict which maps name: df
dfs = {}
for x in os.listdir(ROOT):
    x_path = os.path.join(ROOT, x)
    if not os.path.isdir(x_path):
        continue
    print(x)
    for y in os.listdir(x_path):
        y_path = os.path.join(x_path, y)
        if not os.path.isdir(y_path):
            continue
        print('----'+y)
        for z in os.listdir(y_path):
            z_path = os.path.join(y_path, z)
            if not z.endswith('csv') or 'meta' in z:
                continue
            print('--------' + z)
            dfs[x] = pd.read_csv(z_path, low_memory=False)

race
----ACSDT5Y2018.B02001_2020-10-11T113117
--------ACSDT5Y2018.B02001_data_with_overlays_2020-10-11T112948.csv
acs_demographic_and_housing_estimates
----ACSDP5Y2018.DP05_2020-10-11T112022
--------ACSDP5Y2018.DP05_data_with_overlays_2020-10-11T111820.csv
employment_status
----ACSST5Y2018.S2301_2020-10-11T112308
--------ACSST5Y2018.S2301_data_with_overlays_2020-10-11T112123.csv
educational_attainment
----ACSST5Y2018.S1501_2020-10-11T112704
--------ACSST5Y2018.S1501_data_with_overlays_2020-10-11T112246.csv
.ipynb_checkpoints
age_and_sex
----ACSST5Y2018.S0101_2020-10-11T112952
--------ACSST5Y2018.S0101_data_with_overlays_2020-10-11T112700.csv
households_and_families
----ACSST5Y2018.S1101_2020-10-11T113434
--------ACSST5Y2018.S1101_data_with_overlays_2020-10-11T113113.csv
income_and_poverty
----ACSST5Y2018.S1701_2020-10-11T110717
--------ACSST5Y2018.S1701_data_with_overlays_2020-10-11T110149.csv
industry_by_occupation
----ACSST5Y2018.S2405_2020-10-11T113601
--------ACSST5Y2018.S2405_data

In [5]:
dfs.keys()

dict_keys(['race', 'acs_demographic_and_housing_estimates', 'employment_status', 'educational_attainment', 'age_and_sex', 'households_and_families', 'income_and_poverty', 'industry_by_occupation'])

In [6]:
def prepare_df(df):
    """
    prepare data.census.gov downloads
    returns prepared df and data dictionary df
    """
    # remove columns that end with M (remove margin of error fields)
    n_cols_before = df.shape[1]
    df = df[[x for x in df.columns if not x.endswith('M')]]
    n_cols_after = df.shape[1]
    assert n_cols_after == n_cols_before / 2 + 1 # make sure we arent dropping anything unexpected
    #display(df.head(3))
    
    # create data dict from columns and first row
    data_dict = df.iloc[0].reset_index()
    data_dict.columns = ['column', 'description']
    #display(data_dict)
    
    # remove first row with descriptions and last row with total US summary data
    df = df.iloc[1:-1]
    #display(df.head(3))
    
   
    # create GEOID as the tail portion of GEO_ID
    df['GEOID'] = df['GEO_ID'].str.split('US').str[-1]
    df.GEOID = pd.to_numeric(df.GEOID, errors='coerce', downcast='integer')
    data_dict = pd.concat([pd.DataFrame({'column': ['GEOID'], 'description': ['Field to Join Gazetteer Data']}), data_dict]).reset_index(drop=True)
    
    if df[df.GEOID.isnull()].shape[0] > 1: # 1 will be made for "US" record, that is ok
        print('WARNING: GEOID parsing created {} nulls.'.format(df[df.GEOID.isnull()].shape[0]))
        
    data_cols = list(set(df.columns)-set(['GEOID', 'GEO_ID', 'NAME']))
    
    # convert data fields to numeric type
    for col in data_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    # reorder columns
    df = df[['GEOID', 'GEO_ID', 'NAME']+data_cols]
    
    return df, data_dict

In [7]:
data_dicts = {}
for name, df in dfs.items():
    print(name)
    dfs[name], data_dicts[name] = prepare_df(df)

race
acs_demographic_and_housing_estimates
employment_status
educational_attainment
age_and_sex
households_and_families
income_and_poverty
industry_by_occupation


In [8]:
display(dfs['race'].head(2))
display(data_dicts['race'])

,GEOID,GEO_ID,NAME,B02001_006E,B02001_005E,B02001_001E,B02001_002E,B02001_009E,B02001_010E,B02001_008E,B02001_004E,B02001_007E,B02001_003E
1,10003000400,1400000US10003000400,"Census Tract 4, New Castle County, Delaware",0,0,2933,1372,0,40,40,26,0,1495
2,10003001300,1400000US10003001300,"Census Tract 13, New Castle County, Delaware",0,110,3512,3223,0,52,52,0,0,127


,column,description
0,GEOID,Field to Join Gazetteer Data
1,GEO_ID,id
2,NAME,Geographic Area Name
3,B02001_001E,Estimate!!Total
4,B02001_002E,Estimate!!Total!!White alone
5,B02001_003E,Estimate!!Total!!Black or African American alone
6,B02001_004E,Estimate!!Total!!American Indian and Alaska Na...
7,B02001_005E,Estimate!!Total!!Asian alone
8,B02001_006E,Estimate!!Total!!Native Hawaiian and Other Pac...
9,B02001_007E,Estimate!!Total!!Some other race alone


In [9]:
# concat data dicts
DD = pd.concat(list(data_dicts.values())).drop_duplicates(keep='first').reset_index(drop=True)

DD

,column,description
0,GEOID,Field to Join Gazetteer Data
1,GEO_ID,id
2,NAME,Geographic Area Name
3,B02001_001E,Estimate!!Total
4,B02001_002E,Estimate!!Total!!White alone
...,...,...
1311,S2405_C03_006E,Estimate!!Service occupations!!Civilian employ...
1312,S2405_C04_006E,Estimate!!Sales and office occupations!!Civili...
1313,S2405_C05_006E,"Estimate!!Natural resources, construction, and..."
1314,S2405_C06_006E,"Estimate!!Production, transportation, and mate..."


In [10]:
# join tables
EST = None
for name, table in dfs.items():
    print(name)
    if EST is None:
        EST = table
        continue
    
    print("Before: {}".format(EST.shape))
    EST = EST.merge(table, on=['NAME', 'GEO_ID', 'GEOID'], how='left')    
    print("After: {}".format(EST.shape))
    

race
acs_demographic_and_housing_estimates
Before: (74001, 13)
After: (74001, 191)
employment_status
Before: (74001, 191)
After: (74001, 331)
educational_attainment
Before: (74001, 331)
After: (74001, 715)
age_and_sex
Before: (74001, 715)
After: (74001, 943)
households_and_families
Before: (74001, 943)
After: (74001, 1043)
income_and_poverty
Before: (74001, 1043)
After: (74001, 1226)
industry_by_occupation
Before: (74001, 1226)
After: (74001, 1316)


In [11]:
EST

GEOID                GEO_ID  \
0      10003000400  1400000US10003000400   
1      10003001300  1400000US10003001300   
2      10003002600  1400000US10003002600   
3      10003010200  1400000US10003010200   
4      10003010300  1400000US10003010300   
...            ...                   ...   
73996  53063010304  1400000US53063010304   
73997  53063010501  1400000US53063010501   
73998  53063010601  1400000US53063010601   
73999  53073000700  1400000US53073000700   
74000  53073001100  1400000US53073001100   

                                                  NAME  B02001_006E  \
0          Census Tract 4, New Castle County, Delaware            0   
1         Census Tract 13, New Castle County, Delaware            0   
2         Census Tract 26, New Castle County, Delaware            0   
3        Census Tract 102, New Castle County, Delaware            0   
4        Census Tract 103, New Castle County, Delaware            0   
...                                                ...          ...   
73996  Census Tract 103.04, Spokane County, Washington            0   
73997  Census Tract 105.01, Spokane County, Washington            0   
73998  Census Tract 106.01, Spokane County, Washington            0   
73999       Census Tract 7, Whatcom County, Washington            0   
74000      Census Tract 11, Whatcom County, Washington           12   

       B02001_005E  B02001_001E  B02001_002E  B02001_009E  B02001_010E  \
0                0         2933         1372            0           40   
1              110         3512         3223            0           52   
2               38         3512         1036            0           64   
3               34         2157         1530            5           22   
4              253         3220         2138            0            0   
...            ...          ...          ...          ...          ...   
73996           39         5857         5753            0           65   
73997          219         8554         7782            0          271   
73998           41         3921         3483           61          217   
73999          552         6581         4847           59          301   
74000          298         6957         6380           54          172   

       B02001_008E  B02001_004E  B02001_007E  B02001_003E  DP05_0046E  \
0               40           26            0         1495           0   
1               52            0            0          127          11   
2               64            0          224         2150           0   
3               27            0           33          533           0   
4                0            0            0          829          28   
...            ...          ...          ...          ...         ...   
73996           65            0            0            0           0   
73997          271          222           55            5           0   
73998          278           48           24           47           0   
73999          360           69          554          199          37   
74000          226            0           25           16          89   

       DP05_0089PE  DP05_0062PE  DP05_0018E  DP05_0044E  DP05_0057E  \
0             53.8          0.0        45.8           0           0   
1             55.6          0.0        50.4         110           0   
2             60.0          0.0        34.2          38         224   
3             48.1          0.0        46.4          34          33   
4             55.9          0.0        38.3         253           0   
...            ...          ...         ...         ...         ...   
73996         51.0          0.0        46.6          39           0   
73997         53.2          0.0        36.7         219          55   
73998         53.8          0.0        44.3          41          24   
73999         49.6          0.0        30.2         552         554   
74000         44.5          0.0        47.7         298          25   

       DP05_

In [12]:
len(set(gaz.GEOID).intersection(set(EST.GEOID))) # expected 74001

74001

In [13]:
EST.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74001 entries, 0 to 74000
Columns: 1316 entries, GEOID to S2405_C02_012E
dtypes: float64(942), int64(372), object(2)
memory usage: 743.6+ MB


In [14]:
DD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316 entries, 0 to 1315
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   column       1316 non-null   object
 1   description  1316 non-null   object
dtypes: object(2)
memory usage: 20.7+ KB


In [15]:
gaz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 8 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   USPS                                                                                                                                    74001 non-null  object 
 1   GEOID                                                                                                                                   74001 non-null  int64  
 2   ALAND                                                                                                                                   74001 non-null  int64  
 3   AWATER                                                        

In [16]:
EST.to_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_ESTIMATES.pkl'), protocol=4)
DD.to_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_ESTIMATES_DD.pkl'), protocol=4)
gaz.to_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_GAZ.pkl'), protocol=4)